# File for plotting

This notebook aims at containing the code for plotting nice pictures for papers using EnergyCommunity.jl

In [ ]:
using Pkg
Pkg.activate(".")

In [ ]:
# parent_dir = "C:/Users/Davide/Downloads/results_DEC/results_paper_NC" #git/gitdf/EnergyCommunity.jl/run_cloud"
parent_dir = "results_final_15_3_2023/results_paper_NC"
saveiter_dir = "$parent_dir/iter"
saveenum_dir = "$parent_dir/enum"

folder_imgs = "$parent_dir/imgs"
folder_imgs_comp = "$folder_imgs/comput"
folder_imgs_leastcore = "$folder_imgs/leastcore"
folder_imgs_reward = "$folder_imgs/reward"
folder_imgs_benefit = "$folder_imgs/benefit"

n_base_users=10
add_EC=true

id_filter = [7, 11, 14, 15]

In [ ]:
using EnergyCommunity
using FileIO
using HiGHS, Plots, StatsPlots, CategoricalArrays
using JuMP
using Gurobi
using TheoryOfGames
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings
using YAML
using CSV

mkpath(folder_imgs)
mkpath(folder_imgs_comp)
mkpath(folder_imgs_leastcore)
mkpath(folder_imgs_reward)
mkpath(folder_imgs_benefit)

fontsize = 1
fontname = "times"

gr()

# default(
#     titlefontsize=20,
#     # tickfontsize=fontsize-2,
#     guidefontsize=fontsize-2,
#     legend_title_font_pointsize=fontsize-1,
#     labelfontsize=fontsize-2,
#     legendfontsize=fontsize-2,
# )

In [ ]:
f_userlist(add_EC, n_users) = [add_EC ? [EC_CODE] : String[]; ["user$u" for u=1:n_users]]

## Store the data of the simulations

#### Enumerative configurations

In [ ]:
EC_size_list_enum = []
dict_enums = Dict()

for filename in readdir(saveenum_dir)
    if endswith(filename, ".jld2") && startswith(filename, "enum_simulations_results_")
        size_enum = parse(Int, replace(filename, "enum_simulations_results_"=>"", ".jld2"=>""))
        push!(EC_size_list_enum, size_enum)
        dict_enums[size_enum] = load("$saveenum_dir/enum_simulations_results_$size_enum.jld2")
    end
end

#### Iterative configurations

In [ ]:
df_iter_options = CSV.read("$saveiter_dir/options_backup.csv", DataFrame)
df_iter_options[!, :id_run] = 1:nrow(df_iter_options)

dict_iter = Dict()
n_iter = 0

while isfile("$saveiter_dir/iter_simulations_results_$(n_iter+1).jld2")
    n_iter += 1
    dict_iter[n_iter] = load("$saveiter_dir/iter_simulations_results_$n_iter.jld2")
end

## Create comparison of computational time

#### Prepare enumerative data

In [ ]:
header_df_time = ["EC_size"]#, "title"]

df_time_enum = DataFrame()

for EC_size in EC_size_list_enum
    df_time_temp = DataFrame(dict_enums[EC_size]["df_time_enum"])  # dict_time_enum

    # df_time_temp[!, setdiff(names(df_time_temp), ["name", "id_run", "EC_size"])] ./= 3600  # change units to hours

    # df_time_temp[!, "title"] = [L"Time [h]"]
    df_time_temp = df_time_temp[!, [header_df_time; setdiff(names(df_time_temp), header_df_time)]]

    if nrow(df_time_enum) == 0
        df_time_enum = df_time_temp
    else
        df_time_enum = vcat(df_time_enum, df_time_temp)
    end
end

df_time_enum

#### Prepare iterative data

In [ ]:
df_history_iter = DataFrame()

for id_run in id_filter #1:n_iter
    df_history_temp = deepcopy(dict_iter[id_run]["df_history"])
    df_history_temp[!, "id_run"] .= id_run

    df_history_temp = df_history_temp[!, ["id_run"; setdiff(names(df_history_temp), ["id_run"])]]
    
    replace!(df_history_temp[!, :benefit_coal], NaN=>0.0)

    if nrow(df_history_iter) == 0
        df_history_iter = df_history_temp
    else
        df_history_iter = vcat(df_history_iter, df_history_temp)
    end
end
header_cols = ["name", "id_run"]
df_history_iter = df_history_iter[!, [header_cols; setdiff(names(df_history_iter), header_cols)]]

# add elapsed time to history dataframe

sort!(df_history_iter, [:id_run, :name])

df_history_iter[!, :iteration_time] .= NaN

groups = groupby(df_history_iter, [:id_run, :name])

for (grp_key, grp) in zip(keys(groups), groups)
    grp[!, :iteration_time] = [grp[1, :elapsed_time]; diff(grp[!, :elapsed_time])]
end

# add gap
df_history_iter[!, :gap] = df_history_iter[!, :value_min_surplus] .- df_history_iter[!, :lower_problem_min_surplus]
df_history_iter[!, :log10_gap] = map(x-> (x<1.0 ? 0.0 : log10(x)), df_history_iter[!, :gap]);

#### Preliminary definitions for plotting

In [ ]:
title_plot = Dict(
    "incore_iter" => "Core",
    "leastcore_iter" => "LeastCore",
    "varleastcore_iter" => "VarLeastCore",
    "varcore_iter" => "VarCore",
);
title_plot_general = Dict(
    "incore_iter" => "IT - Core",
    "leastcore_iter" => "IT - LeastCore",
    "varleastcore_iter" => "IT - VarLeastCore",
    "varcore_iter" => "IT - VarCore",
    "shapley_enum" => "EN - Shapley",
    "nucleolus_enum" => "EN - Nucleolus",
    "mode_time" => "EN - Base",
    "incore_enum" => "EN - Core",
    "leastcore_enum" => "EN - LeastCore",
    "varleastcore_enum" => "EN - VarLeastCore",
    "varcore_enum" => "EN - VarCore",
);

plot_xlims_iters = [0., 50.]
plot_xlims_time = [0., 10.]
plot_ylims = [0., 6.]

rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])

#### Plot total computational time

In [ ]:
cols = Symbol.([string(n) for n in names(df_time_enum) if endswith(string(n), "_enum")])
df_time_enum_sel = stack(df_time_enum, cols)
select!(df_time_enum_sel, Not([:name, :mode_time]))
rename!(df_time_enum_sel, :variable=>:name, :value=>:time)

df_time_iter_sel = combine(groupby(df_history_iter,[:name, :id_run]), :elapsed_time => maximum => :time)
df_time_iter_sel[!, :EC_size] = map(x->df_iter_options[x, :EC_size], df_time_iter_sel[!, :id_run])

df_time_tot = vcat(df_time_iter_sel, df_time_enum_sel)
select!(df_time_tot, Not(:id_run))

first(df_time_tot, 5)

In [ ]:
df_time_unstacked = unstack(df_time_tot, :EC_size, :time)
df_time_unstacked[!, :name] = map(x->title_plot_general[x], df_time_unstacked[!, :name])
for col in eachcol(df_time_unstacked)
    replace!(col, missing=>NaN)
end

df_time_unstacked[!, 2:end] ./= 3600  # convert from seconds to hours

# filter!(
#     x->(contains(x[:name], "Core") | contains(x[:name], "Shapley")), df_time_unstacked
# )

n_functions = length(unique(df_time_unstacked[!, :name]))
n_ec_sizes = length(unique(df_time_tot[!, :EC_size]))

dataplot = Matrix{Float64}(df_time_unstacked[!, 2:end])
ecsize_names = repeat(sort(string.(unique(df_time_tot[!, :EC_size]))), inner=n_functions)
funnames = repeat(unique(df_time_unstacked[!, :name]), outer=n_ec_sizes)

p_total_time = groupedbar(
    ecsize_names,
    (dataplot),
    group=funnames,
    framestyle=:box,
    xlabel="EC size",
    ylabel="Time [h]",
    # ylims=[2900, 3300],
)

display(p_total_time)

savefig(p_total_time, "$folder_imgs/total_time_map.pdf")

In [ ]:
df_time_unstacked

#### Plot of computational time by iteration

In [ ]:
for ((k,), v) in pairs(groupby(df_history_iter, [:name]))
    # initialize plot
    p = plot(title=title_plot[k])

    # loop over run ids
    for ((k_r,), v_r) in pairs(groupby(v, [:id_run]))
        plot!(
            p,
            v_r[!, :iteration],
            v_r[!, :log10_gap],
            label="$(df_iter_options[k_r, :EC_size])",
            legendtitle="EC size",
            xlims=plot_xlims_iters,
            ylims=plot_ylims,
        )
    end

    xlabel!("Iteration [#]")
    ylabel!("log10 gap [-]")

    if k in ["incore_iter", "varcore_iter"]
        hline!([1], label=nothing, color=:black, linestyle=:dash)
    else
        plot!(rectangle(plot_xlims_iters[2],log10(20)-log10(10),0,log10(10)), color=:black, opacity=.2, label=nothing)
    end
    annotate!(230, 1.03, text("Convergence tolerance", :black, :right, :bottom, 9))

    display(p)

    savefig(p, "$folder_imgs_comp/gap_iters_$k.pdf")
end

In [ ]:
for ((k,), v) in pairs(groupby(df_history_iter, [:name]))
    # initialize plot
    p = plot(title=title_plot[k])

    # loop over run ids
    for ((k_r,), v_r) in pairs(groupby(v, [:id_run]))
        plot!(
            p,
            v_r[!, :elapsed_time] ./3600,
            v_r[!, :log10_gap],
            label="$(df_iter_options[k_r, :EC_size])",
            legendtitle="EC size",
            xlims=plot_xlims_time,
            ylims=plot_ylims,
        )
    end

    # if k ∈ ["incore_iter", "leastcore_iter", "varcore_iter", "varleastcore_iter"]
    #     for row in eachrow(df_time_enum)
    #         hline!(
    #             p,
    #             [row[replace(k, "_iter"=>"_enum")]],
    #             label="ENUM - $(row[:EC_size])",
    #             linestyle=:dash,
    #         )
    #     end
    # end

    xlabel!("Elapsed time [h]")
    ylabel!("log10 gap [-]")

    if k in ["incore_iter", "varcore_iter"]
        hline!([1], label=nothing, color=:black, linestyle=:dash)
    else
        plot!(rectangle(plot_xlims_iters[2],log10(20)-log10(10),0,log10(10)), color=:black, opacity=.2, label=nothing)
    end
    annotate!(plot_xlims_iters[2], 1.03, text("Convergence tolerance", :black, :right, :bottom, 9))

    display(p)

    savefig(p, "$folder_imgs_comp/gap_time_$k.pdf")
end

## Least core versus size

In [ ]:
functions_to_consider = ["leastcore_iter", "varleastcore_iter"]

history_selection = filter(x->(x.name ∈ functions_to_consider), df_history_iter)
history_sel_last = combine(groupby(history_selection, [:name, :id_run])) do sdf
    sdf[argmax(sdf.iteration), :]
end

df_merged = innerjoin(
    history_sel_last[:, [:name, :id_run, :value_min_surplus, :lower_problem_min_surplus]],
    df_iter_options[!, [:id_run, :EC_size]],
    on=[:id_run],
)[!, [:name, :EC_size, :value_min_surplus]]

sort!(df_merged, [:name, :EC_size])

In [ ]:
df_unstacked = unstack(df_merged, :EC_size, :value_min_surplus)
dataplot = Matrix{Float64}(df_unstacked[!, 2:end])
ecsize_names = repeat(sort([string(df_iter_options[r, :EC_size]) for r in id_filter]), inner=length(functions_to_consider))
funnames = repeat([title_plot[f] for f in functions_to_consider], outer=length(id_filter))

p_leastcore = groupedbar(
    ecsize_names,
    dataplot,
    group=funnames,
    framestyle=:box,
    xlabel="EC size",
    ylabel="LeastCore [€]",
    ylims=[2900, 3300],
)

display(p_leastcore)

savefig(p_leastcore, "$folder_imgs_leastcore/leastcore_vs_size.pdf");

## Benefit distribution versus size

#### Store data of reward distribution

In [ ]:
df_reward_iter = DataFrame()

largest_user_set = []

for id_run in id_filter
    df_benefit_temp = deepcopy(dict_iter[id_run]["df_reward_iter"])
    df_benefit_temp[!, "EC_size"] .= nrow(df_benefit_temp)-1
    df_benefit_temp[!, "id_run"] .= id_run

    # Put users as columns
    df_benefit_temp = unstack(stack(df_benefit_temp), :user_set, :value)
    if nrow(df_reward_iter) == 0
        df_reward_iter = df_benefit_temp
    else
        df_reward_iter = vcat(df_reward_iter, df_benefit_temp, cols=:union)
    end
end

# sort by user set
df_reward_iter = df_reward_iter[!, ["variable"; "EC_size"; "id_run"; EC_CODE; ["user$i" for i = 1:(ncol(df_reward_iter)-4)]]]
first(df_reward_iter, 5)

#### utils for benefit computations

In [ ]:
# custom mean, maximum and minimum function handling missing values
_mean(x) = sum(sum(skipmissing(x)))/count(!ismissing, x)
_maximum = maximum ∘ skipmissing
_minimum = minimum ∘ skipmissing

# function to automatically create matrix blocks for aggregation on the data
function draw_stats(f, df, n_base_users=n_base_users, add_EC=add_EC)
    reward_EC = add_EC ? df[:, [EC_CODE]] : DataFrame()

    df_list = []
    for u=1:n_base_users
    
        sel = df[!, ["user$uu" for uu in u:n_base_users:maximum(df[!, :EC_size])]]
    
        push!(df_list, combine(sel, AsTable(:) => ByRow(f) => "user$u"))
    end
    
    df_out = hcat(reward_EC, df_list...)

    return Matrix{Float64}(df_out)
end

#### Plot benefit distribution with increasing size community

In [ ]:
user_focus = f_userlist(add_EC, 10)

for ((k,), v) in pairs(groupby(df_reward_iter, [:variable]))

    df_benefit_selection = sort(v, [:EC_size])

    mean_data = draw_stats(_mean, df_benefit_selection)' ./ 1000
    max_data = draw_stats(_maximum, df_benefit_selection)' ./ 1000
    min_data = draw_stats(_minimum, df_benefit_selection)' ./ 1000

    max_err = max_data - mean_data
    min_err = mean_data - min_data

    grp_ecsize = repeat(string.(sort(collect(v[!, :EC_size]))), inner=length(user_focus))
    grp_users = CategoricalArray(repeat(user_focus, outer=length(id_filter)))
    levels!(grp_users, user_focus)

    p = groupedbar(
        grp_users,
        mean_data,
        group=grp_ecsize,
        framestyle=:box,
        legendtitle="EC size",
        xlabel="User/EC",
        ylabel="Net benefit [k€]",
        title=title_plot[k],
        yerror=(min_err, max_err),
        # legend=:outerright,
        # ylims=[0, 100]
    )

    display(p)
    
    savefig(p, "$folder_imgs_benefit/benefit_vs_size_$k.pdf");
end

## Reward distribution versus size

#### Load the solved networks for each size of community

In [ ]:
ec_models = Dict(
    ec_size=>load!("$parent_dir/iter/ec_model_$ec_size.jld2", ModelEC())
    for ec_size in unique(df_reward_iter[!, :EC_size])
)

nc_models = Dict(
    ec_size=>load!("$parent_dir/iter/nc_model_$ec_size.jld2", ModelEC())
    for ec_size in unique(df_reward_iter[!, :EC_size])
)

total_reward_by_EC = Dict(
    ec_size=>ecm.results[:R_Reward_agg_NPV] for (ec_size, ecm) in ec_models
)

#### Calculate shared reward for each user

In [ ]:
all_users = f_userlist(add_EC, maximum(df_iter_options[!, :EC_size]))

function get_split_terms(ec_models, nc_models, id_run, data)
    ec_size = df_iter_options[id_run, :EC_size]
    ecm = ec_models[ec_size]
    ncm = nc_models[ec_size]
    ulist = f_userlist(add_EC, ec_size)
    NPV_tot = Dict(
        uname=> (uname==EC_CODE) ? 0.0 : ncm.results[:NPV_us][uname] for uname in ulist
    )
    profit_dist = Dict(uname=>data[Symbol(uname)] + NPV_tot[uname] for uname in ulist)

    return split_financial_terms(ecm, profit_dist)
end

function reward_terms(ec_models, nc_models, id_run, data, all_user_list)
    ec_size = df_iter_options[id_run, :EC_size]
    ulist = f_userlist(add_EC, ec_size)
    reward = get_split_terms(ec_models, nc_models, id_run, data).REWARD

    return [((u in ulist) ? reward[u] : missing) for u in all_user_list]
end

df_sharedreward_iter = combine(
    df_reward_iter,
    AsTable(:) => ByRow(x-> [
        values(x[[:variable, :id_run, :EC_size]])...;
        reward_terms(ec_models, nc_models, x.id_run, x, all_users)
    ]) => [:variable; :id_run; :EC_size; Symbol.(all_users)]
);

#### Plot reward allocation across users

In [ ]:
user_focus = f_userlist(add_EC, 10)

for ((k,), v) in pairs(groupby(df_sharedreward_iter, [:variable]))

    df_shared = sort(v, [:EC_size])

    mean_data = draw_stats(_mean, df_shared)' ./ 1000
    max_data = draw_stats(_maximum, df_shared)' ./ 1000
    min_data = draw_stats(_minimum, df_shared)' ./ 1000

    max_err = max_data - mean_data
    min_err = mean_data - min_data

    grp_ecsize = repeat(string.(sort(collect(v[!, :EC_size]))), inner=length(user_focus))
    grp_users = CategoricalArray(repeat(user_focus, outer=length(id_filter)))
    levels!(grp_users, user_focus)

    p = groupedbar(
        grp_users,
        mean_data,
        group=grp_ecsize,
        framestyle=:box,
        legendtitle="EC size",
        xlabel="User/EC",
        ylabel="Reward allocation [k€]",
        title=title_plot[k],
        yerror=(min_err, max_err),
        # legend=:outerright,
        # ylims=[0, 100]
    )

    display(p)
    
    savefig(p, "$folder_imgs_reward/reward_vs_size_$k.pdf");
end